- [Dog vs cat in Kaggle](https://www.kaggle.com/c/dogs-vs-cats)
- [Original note book](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/lesson1.ipynb)


## Basic Setup
Load library

In [1]:
%run env_setup.py
from importlib import reload
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

Load data

In [2]:
from utils.datasets.kaggle import dog_cat
reload(dog_cat)

data_dir = dog_cat.download_data()

Default path

In [3]:
#path = os.path.join(data_dir, 'sample')
path = data_dir

## Use pretrained VGG model

- VGG19: large, slow, slightly accurate
- VGG16: small, fast

We use VGG16

In [4]:
## As large as you can, but no larger than 64 is recommanded
batch_size = 64

In [5]:
import vgg16
reload(vgg16)
from vgg16 import Vgg16

Using TensorFlow backend.


In [7]:
vgg = Vgg16()
batches = vgg.get_batches(os.path.join(path, 'train'), batch_size=batch_size)
val_batches = vgg.get_batches(os.path.join(path, 'valid'), batch_size=batch_size)

vgg.finetune(batches)
vgg.fit(batches, val_batches, nb_epoch=1)

D:\learn\online_course\notes\fast_ai\deeplearning1\vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
D:\learn\online_course\notes\fast_ai\deeplearning1\vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
D:\learn\online_course\notes\fast_ai\deeplearning1\vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
D:\learn\online_course\notes\fast_ai\deeplearning1\vgg16.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/1
359/359 [==============================] - 151s - loss: 0.4408 - acc: 0.8691 - val_loss: 0.2688 - val_acc: 0.9047


## VGG16 details

In [8]:
imgs, labels = batches.next()

In [10]:
print(imgs.shape)

(64, 3, 224, 224)
